In [9]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import osmnx as ox

import geopandas as gpd
from shapely.geometry import Point, LineString, MultiLineString
from math import radians, sin, cos, sqrt, atan2
import osmnx as ox

#from mpl_toolkits.basemap import Basemap as Basemap

In [77]:
lista_estaciones = pd.read_csv('Avances/lista_estaciones.csv')
df_lista_estaciones = lista_estaciones.copy()

viajes = pd.read_csv('Avances/viajes_procesado.csv')
df_viajes = viajes.copy()

In [78]:
df_viajes['dia_origen_fecha'] = pd.to_datetime(df_viajes['dia_origen_fecha'])
df_viajes['horario_origen'] = pd.to_datetime(df_viajes['horario_origen'], format='%H:%M').dt.time

df_viajes['datetime_origen'] = pd.to_datetime(df_viajes['dia_origen_fecha'].astype(str) + ' ' + df_viajes['horario_origen'].astype(str))

# Repeat the same steps for 'dia_destino_fecha' and 'horario_destino'
df_viajes['dia_destino_fecha'] = pd.to_datetime(df_viajes['dia_destino_fecha'])
df_viajes['horario_destino'] = pd.to_datetime(df_viajes['horario_destino'], format='%H:%M').dt.time

df_viajes['datetime_destino'] = pd.to_datetime(df_viajes['dia_destino_fecha'].astype(str) + ' ' + df_viajes['horario_destino'].astype(str))

In [79]:
df_viajes

,Id_recorrido,duracion_recorrido,dia_origen_fecha,dia_semana_origen,horario_origen,id_estacion_origen,nombre_estacion_origen,dia_destino_fecha,dia_semana_destino,horario_destino,id_estacion_destino,nombre_estacion_destino,id_usuario,modelo_bicicleta,Género,tipo_viaje,datetime_origen,datetime_destino
0,13267975,2610,2022-01-16,1,14:58:00,5.0,Plaza Italia,2022-01-16,1,15:42:00,335.0,General Urquiza,776361,ICONIC,FEMALE,PAGO,2022-01-16 14:58:00,2022-01-16 15:42:00
1,13268526,545,2022-01-16,1,17:26:00,51.0,Tucuman,2022-01-16,1,17:35:00,174.0,Ministerio De Educacion,776407,ICONIC,MALE,GRATUITO,2022-01-16 17:26:00,2022-01-16 17:35:00
2,13268400,2061,2022-01-16,1,16:51:00,161.0,Humahuaca,2022-01-16,1,17:25:00,117.0,Humberto 1°,671762,ICONIC,FEMALE,PAGO,2022-01-16 16:51:00,2022-01-16 17:25:00
3,13268164,12748,2022-01-16,1,15:58:00,335.0,General Urquiza,2022-01-16,1,19:30:00,204.0,Biarritz,776361,ICONIC,FEMALE,PAGO,2022-01-16 15:58:00,2022-01-16 19:30:00
4,13270010,4337,2022-01-16,1,23:40:00,113.0,Guatemala,2022-01-17,2,00:52:00,125.0,F.j.santamaria De Oro,454615,ICONIC,OTHER,PAGO,2022-01-16 23:40:00,2022-01-17 00:52:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2922797,16700616,734,2022-12-20,3,22:34:00,207.0,Plaza Arenales,2022-12-20,3,22:46:00,224.0,Baigorria,887601,FIT,FEMALE,GRATUITO,2022-12-20 22:34:00,2022-12-20 22:46:00
2922798,16699813,764,2022-12-20,3,20:03:00,390.0,De Los Ombues,2022-12-20,3,20:16:00,15.0,Amenabar,729879,FIT,MALE,GRATUITO,2022-12-20 20:03:00,2022-12-20 20:16:00
2922799,16695096,4328,2022-12-20,3,12:13:00,169.0,Facultad De Psicologia,2022-12-20,3,13:25:00,175.0,Plaza Mackenna,220076,FIT,FEMALE,PAGO,2022-12-20 12:13:00,2022-12-20 13:25:00
2922800,16698330,1405,2022-12-20,3,17:26:00,62.0,Parque Onega,2022-12-20,3,17:50:00,234.0,Avenida Triunvirato,193874,FIT,MALE,GRATUITO,2022-12-20 17:26:00,2022-12-20 17:50:00


In [80]:
df_viajes

,Id_recorrido,duracion_recorrido,dia_origen_fecha,dia_semana_origen,horario_origen,id_estacion_origen,nombre_estacion_origen,dia_destino_fecha,dia_semana_destino,horario_destino,id_estacion_destino,nombre_estacion_destino,id_usuario,modelo_bicicleta,Género,tipo_viaje,datetime_origen,datetime_destino
0,13267975,2610,2022-01-16,1,14:58:00,5.0,Plaza Italia,2022-01-16,1,15:42:00,335.0,General Urquiza,776361,ICONIC,FEMALE,PAGO,2022-01-16 14:58:00,2022-01-16 15:42:00
1,13268526,545,2022-01-16,1,17:26:00,51.0,Tucuman,2022-01-16,1,17:35:00,174.0,Ministerio De Educacion,776407,ICONIC,MALE,GRATUITO,2022-01-16 17:26:00,2022-01-16 17:35:00
2,13268400,2061,2022-01-16,1,16:51:00,161.0,Humahuaca,2022-01-16,1,17:25:00,117.0,Humberto 1°,671762,ICONIC,FEMALE,PAGO,2022-01-16 16:51:00,2022-01-16 17:25:00
3,13268164,12748,2022-01-16,1,15:58:00,335.0,General Urquiza,2022-01-16,1,19:30:00,204.0,Biarritz,776361,ICONIC,FEMALE,PAGO,2022-01-16 15:58:00,2022-01-16 19:30:00
4,13270010,4337,2022-01-16,1,23:40:00,113.0,Guatemala,2022-01-17,2,00:52:00,125.0,F.j.santamaria De Oro,454615,ICONIC,OTHER,PAGO,2022-01-16 23:40:00,2022-01-17 00:52:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2922797,16700616,734,2022-12-20,3,22:34:00,207.0,Plaza Arenales,2022-12-20,3,22:46:00,224.0,Baigorria,887601,FIT,FEMALE,GRATUITO,2022-12-20 22:34:00,2022-12-20 22:46:00
2922798,16699813,764,2022-12-20,3,20:03:00,390.0,De Los Ombues,2022-12-20,3,20:16:00,15.0,Amenabar,729879,FIT,MALE,GRATUITO,2022-12-20 20:03:00,2022-12-20 20:16:00
2922799,16695096,4328,2022-12-20,3,12:13:00,169.0,Facultad De Psicologia,2022-12-20,3,13:25:00,175.0,Plaza Mackenna,220076,FIT,FEMALE,PAGO,2022-12-20 12:13:00,2022-12-20 13:25:00
2922800,16698330,1405,2022-12-20,3,17:26:00,62.0,Parque Onega,2022-12-20,3,17:50:00,234.0,Avenida Triunvirato,193874,FIT,MALE,GRATUITO,2022-12-20 17:26:00,2022-12-20 17:50:00


In [81]:
df_lista_estaciones

# Add a new column with lat-lon tuples
df_lista_estaciones['lat_lon'] = df_lista_estaciones.apply(lambda row: Point( row['long_estacion_origen'],row['lat_estacion_origen']), axis=1)
df_lista_estaciones

,nombre_estacion_origen,id_estacion_origen,direccion_estacion_origen,long_estacion_origen,lat_estacion_origen,lat_lon
0,Plaza Italia,5.0,Av. Sarmiento 2601,-58.420954,-34.580550,POINT (-58.4209542 -34.5805497)
1,Tucuman,51.0,Tucuman & 9 De Julio Av.,-58.382126,-34.601478,POINT (-58.3821261 -34.6014776)
2,Humahuaca,161.0,3912 Humahuaca,-58.419676,-34.602078,POINT (-58.4196761 -34.6020779)
3,General Urquiza,335.0,Figueroa Alcorta & Sarmiento,-58.411278,-34.572165,POINT (-58.411278 -34.572165)
4,Guatemala,113.0,Guatemala 4773,-58.424996,-34.585878,POINT (-58.424996 -34.585878)
...,...,...,...,...,...,...
326,Allende,314.0,Allende 2794,-58.511672,-34.614787,POINT (-58.5116722 -34.6147875)
327,Cerretti,307.0,Ceretti 2892,-58.496232,-34.574424,POINT (-58.4962318 -34.5744245)
328,Habana,55.0,Gral. José Gervasio Artigas 4298 (y Habana),-58.494959,-34.586598,POINT (-58.4949585 -34.5865976)
329,Belgrano,57.0,5 de julio 423,-58.371250,-34.612690,POINT (-58.37125 -34.6126898)


In [82]:
df_viajes
df_viajes = df_viajes.rename(columns={'nombre_estacion_origen_x': 'nombre_estacion_origen'})

In [83]:
#Agrego al dataframe de viajes los datos de puntos de geopandas del inicio y la llegada

df_viajes = df_viajes.merge(df_lista_estaciones[['nombre_estacion_origen','lat_lon']],on='nombre_estacion_origen',how = 'left')
# Rename the column in the merged dataframe
df_viajes = df_viajes.rename(columns={'lat_lon': 'lat_lon_origen'})


In [84]:
df_viajes = df_viajes.merge(df_lista_estaciones[['nombre_estacion_origen','lat_lon']],left_on='nombre_estacion_destino', right_on='nombre_estacion_origen',how = 'left')
# Rename the column in the merged dataframe
df_viajes = df_viajes.rename(columns={'lat_lon': 'lat_lon_destino'})
df_viajes = df_viajes.rename(columns={'nombre_estacion_origen_x': 'nombre_estacion_origen'})

In [85]:
df_viajes = df_viajes.drop('nombre_estacion_origen_y', axis=1)

In [86]:
df_viajes

,Id_recorrido,duracion_recorrido,dia_origen_fecha,dia_semana_origen,horario_origen,id_estacion_origen,nombre_estacion_origen,dia_destino_fecha,dia_semana_destino,horario_destino,id_estacion_destino,nombre_estacion_destino,id_usuario,modelo_bicicleta,Género,tipo_viaje,datetime_origen,datetime_destino,lat_lon_origen,lat_lon_destino
0,13267975,2610,2022-01-16,1,14:58:00,5.0,Plaza Italia,2022-01-16,1,15:42:00,335.0,General Urquiza,776361,ICONIC,FEMALE,PAGO,2022-01-16 14:58:00,2022-01-16 15:42:00,POINT (-58.4209542 -34.5805497),POINT (-58.411278 -34.572165)
1,13268526,545,2022-01-16,1,17:26:00,51.0,Tucuman,2022-01-16,1,17:35:00,174.0,Ministerio De Educacion,776407,ICONIC,MALE,GRATUITO,2022-01-16 17:26:00,2022-01-16 17:35:00,POINT (-58.3821261 -34.6014776),POINT (-58.391768 -34.597225)
2,13268400,2061,2022-01-16,1,16:51:00,161.0,Humahuaca,2022-01-16,1,17:25:00,117.0,Humberto 1°,671762,ICONIC,FEMALE,PAGO,2022-01-16 16:51:00,2022-01-16 17:25:00,POINT (-58.4196761 -34.6020779),POINT (-58.3741759 -34.6201008)
3,13268164,12748,2022-01-16,1,15:58:00,335.0,General Urquiza,2022-01-16,1,19:30:00,204.0,Biarritz,776361,ICONIC,FEMALE,PAGO,2022-01-16 15:58:00,2022-01-16 19:30:00,POINT (-58.411278 -34.572165),POINT (-58.47739 -34.60547)
4,13270010,4337,2022-01-16,1,23:40:00,113.0,Guatemala,2022-01-17,2,00:52:00,125.0,F.j.santamaria De Oro,454615,ICONIC,OTHER,PAGO,2022-01-16 23:40:00,2022-01-17 00:52:00,POINT (-58.424996 -34.585878),POINT (-58.428016 -34.583323)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2922797,16700616,734,2022-12-20,3,22:34:00,207.0,Plaza Arenales,2022-12-20,3,22:46:00,224.0,Baigorria,887601,FIT,FEMALE,GRATUITO,2022-12-20 22:34:00,2022-12-20 22:46:00,POINT (-58.5111458 -34.5997308),POINT (-58.50302 -34.6108915)
2922798,16699813,764,2022-12-20,3,20:03:00,390.0,De Los Ombues,2022-12-20,3,20:16:00,15.0,Amenabar,729879,FIT,MALE,GRATUITO,2022-12-20 20:03:00,2022-12-20 20:16:00,POINT (-58.4281 -34.5602),POINT (-58.4475236 -34.572056)
2922799,16695096,4328,2022-12-20,3,12:13:00,169.0,Facultad De Psicologia,2022-12-20,3,13:25:00,175.0,Plaza Mackenna,220076,FIT,FEMALE,PAGO,2022-12-20 12:13:00,2022-12-20 13:25:00,POINT (-58.411856 -34.6123459),POINT (-58.4771259 -34.5436888)
2922800,16698330,1405,2022-12-20,3,17:26:00,62.0,Parque Onega,2022-12-20,3,17:50:00,234.0,Avenida Triunvirato,193874,FIT,MALE,GRATUITO,2022-12-20 17:26:00,2022-12-20 17:50:00,POINT (-58.513837 -34.5934185),POINT (-58.4952184 -34.5657087)


In [87]:
estaciones = df_viajes['nombre_estacion_origen'].to_list()

In [88]:
estaciones

[' Plaza Italia',
 ' Tucuman',
 ' Humahuaca',
 ' General Urquiza',
 ' Guatemala',
 ' Garcia Del Rio',
 ' Monroe',
 ' Agronomia',
 ' Biarritz',
 ' Plazoleta Colombia',
 ' Ecoparque',
 ' San Jose De Flores',
 ' Acevedo',
 ' Cerrito',
 ' Carlos Gardel',
 ' Aduana',
 ' Julieta Lanteri',
 ' Lima',
 ' Barrancas De Belgrano',
 ' Coghlan',
 ' Pacifico',
 ' Villa Urquiza',
 ' Beiro',
 ' Plaza Italia',
 ' Garcia Del Rio',
 ' Beiro Y San Martín',
 ' Plazoleta Colombia',
 ' Las Casas',
 ' Plaza Italia',
 ' Plaza Almagro',
 ' Montevideo',
 ' Billinghurst',
 ' Aguero',
 ' Hospital Garrahan',
 ' Macacha Guemes',
 ' Plaza Alemania',
 ' Posadas',
 ' Arenales Y Aguero',
 ' Facultad De Derecho',
 ' F.j.santamaria De Oro',
 ' San Miguel De Garicoits',
 ' Plaza Irlanda',
 ' Madero Office',
 ' Parque Las Heras',
 ' Pacifico',
 ' Juana Manso I',
 ' Uruguay',
 ' Chile',
 ' Billinghurst',
 ' Cerrito',
 ' Paraná',
 ' Saavedra',
 ' Guayaquil',
 ' Esmeralda',
 ' Humberto 1°',
 ' Yatay',
 'Hospital De Clínicas',
 

In [133]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from pandas import Timestamp


def plot_bike_stock(df, initial_date, final_date, station, initial_capacity):
    # Convert initial_date and final_date to pandas Timestamp objects
    inicio_grafico = initial_date
    fin_grafico = final_date
    initial_date = Timestamp(initial_date)
    final_date = Timestamp(final_date)
    # Filter the DataFrame based on initial and final dates
    mask = ((df['datetime_origen'] >= initial_date) & (df['datetime_origen'] <= final_date)) | ((df['datetime_destino'] >= initial_date) & (df['datetime_destino'] <= final_date))
    station_mask = (df['nombre_estacion_origen'] == station) | (df['nombre_estacion_destino'] == station)
    filtered_df = df.loc[mask & station_mask]

    # Initialize a new DataFrame for bike stock information
    stock_df = pd.DataFrame(columns=['datetime', 'bike_stock', 'registry_type'])

    # Iterate through the filtered DataFrame rows and update the bike stock
    for _, row in filtered_df.iterrows():
        origin_station = row['nombre_estacion_origen']
        destination_station = row['nombre_estacion_destino']

        # Update bike stock based on origin and destination stations
        if origin_station == station:
            stock_df = stock_df.append({'datetime': row['datetime_origen'], 'registry_type': -1}, ignore_index=True)
        elif destination_station == station:
            stock_df = stock_df.append({'datetime': row['datetime_destino'], 'registry_type': 1}, ignore_index=True)

    # Sort the DataFrame by datetime
    stock_df.sort_values('datetime', inplace=True)

    # Calculate bike stock
    stock_df['bike_stock'] = initial_capacity + stock_df['registry_type'].cumsum()

    # Create a time series plot with Plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=stock_df['datetime'], y=stock_df['bike_stock'], name="hv",
                             line_shape='hv'))

    fig.update_layout(
        title='Bicicletas disponibles en estación: ' + station+ 'entre el día ' + str(inicio_grafico) + ' y el día '+ str(fin_grafico),
        xaxis=dict(
            title='Datetime',
            tickformat='%Y-%m-%d %H:%M',
            tickangle=45,
            tickfont=dict(size=10),
            zeroline=False,  # Remove the zero line
        ),
        yaxis=dict(
            title='Bike Stock',
            range=[0, 1.25 * initial_capacity]
        ),
        hovermode='x',
        template='plotly_white'
    )

    # Add vertical line annotations to simulate vertical gridlines
    for date in pd.date_range(initial_date, final_date):
        fig.add_shape(
            type='line',
            x0=date,
            x1=date,
            y0=0,
            y1=1.25 * initial_capacity,
            line=dict(color='lightgray', width=1, dash='dash')
        )

    fig.show()

    return filtered_df, stock_df




In [138]:
df_plaza_italia = plot_bike_stock(df_viajes, '2022-06-12', '2022-06-26', ' Constitución', 10)

C:\Users\agust\AppData\Local\Temp\ipykernel_4632\1835637359.py:28: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\agust\AppData\Local\Temp\ipykernel_4632\1835637359.py:30: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\agust\AppData\Local\Temp\ipykernel_4632\1835637359.py:28: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\agust\AppData\Local\Temp\ipykernel_4632\1835637359.py:28: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\agust\AppData\Local\Temp\ipykernel_4632\1835637359.py:30: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

In [124]:
df_plaza_italia[1].head(20)

,datetime,bike_stock,registry_type
64,2022-06-12 00:44:00,21,1
238,2022-06-12 01:06:00,22,1
315,2022-06-12 01:17:00,21,-1
131,2022-06-12 05:52:00,22,1
53,2022-06-12 06:31:00,23,1
186,2022-06-12 13:09:00,22,-1
433,2022-06-12 13:26:00,23,1
130,2022-06-12 13:34:00,22,-1
440,2022-06-12 13:35:00,21,-1
176,2022-06-12 13:44:00,22,1


In [126]:
df_plaza_italia[0].head(20)

,Id_recorrido,duracion_recorrido,dia_origen_fecha,dia_semana_origen,horario_origen,id_estacion_origen,nombre_estacion_origen,dia_destino_fecha,dia_semana_destino,horario_destino,id_estacion_destino,nombre_estacion_destino,id_usuario,modelo_bicicleta,Género,tipo_viaje,datetime_origen,datetime_destino,lat_lon_origen,lat_lon_destino
1159223,14731488,459,2022-06-14,3,08:50:00,33.0,Facultad De Medicina,2022-06-14,3,08:58:00,38.0,Plaza Libertad,252459,ICONIC,FEMALE,GRATUITO,2022-06-14 08:50:00,2022-06-14 08:58:00,POINT (-58.3989807 -34.5970909),POINT (-58.3828403 -34.5970497)
1159234,14737402,846,2022-06-14,3,17:46:00,38.0,Plaza Libertad,2022-06-14,3,18:00:00,116.0,Hospital Alemán,207946,ICONIC,OTHER,GRATUITO,2022-06-14 17:46:00,2022-06-14 18:00:00,POINT (-58.3828403 -34.5970497),POINT (-58.4025894 -34.5921708)
1159374,14759176,3011,2022-06-16,5,16:17:00,38.0,Plaza Libertad,2022-06-16,5,17:07:00,187.0,Jose Maria Moreno,32558,ICONIC,MALE,GRATUITO,2022-06-16 16:17:00,2022-06-16 17:07:00,POINT (-58.3828403 -34.5970497),POINT (-58.435801 -34.619879)
1159392,14760046,1401,2022-06-16,5,17:13:00,137.0,Azopardo Y Chile,2022-06-16,5,17:37:00,38.0,Plaza Libertad,16035,ICONIC,MALE,GRATUITO,2022-06-16 17:13:00,2022-06-16 17:37:00,POINT (-58.3674923 -34.6155977),POINT (-58.3828403 -34.5970497)
1159687,14726525,751,2022-06-13,2,17:55:00,38.0,Plaza Libertad,2022-06-13,2,18:07:00,116.0,Hospital Alemán,207946,ICONIC,OTHER,GRATUITO,2022-06-13 17:55:00,2022-06-13 18:07:00,POINT (-58.3828403 -34.5970497),POINT (-58.4025894 -34.5921708)
1160596,14726069,1064,2022-06-13,2,17:25:00,38.0,Plaza Libertad,2022-06-13,2,17:43:00,25.0,Plaza Guemes,841388,ICONIC,OTHER,GRATUITO,2022-06-13 17:25:00,2022-06-13 17:43:00,POINT (-58.3828403 -34.5970497),POINT (-58.4161178 -34.5894269)
1160598,14725844,597,2022-06-13,2,17:13:00,38.0,Plaza Libertad,2022-06-13,2,17:23:00,35.0,Ingeniero Butty,205517,ICONIC,MALE,GRATUITO,2022-06-13 17:13:00,2022-06-13 17:23:00,POINT (-58.3828403 -34.5970497),POINT (-58.371847 -34.5964246)
1160600,14727705,1060,2022-06-13,2,19:09:00,25.0,Plaza Guemes,2022-06-13,2,19:26:00,38.0,Plaza Libertad,841388,ICONIC,OTHER,GRATUITO,2022-06-13 19:09:00,2022-06-13 19:26:00,POINT (-58.4161178 -34.5894269),POINT (-58.3828403 -34.5970497)
1160601,14719954,569,2022-06-13,2,08:18:00,130.0,Retiro Ii,2022-06-13,2,08:27:00,38.0,Plaza Libertad,205517,ICONIC,MALE,GRATUITO,2022-06-13 08:18:00,2022-06-13 08:27:00,POINT (-58.37436403 -34.5917376),POINT (-58.3828403 -34.5970497)
1160658,14747658,346,2022-06-15,4,16:24:00,38.0,Plaza Libertad,2022-06-15,4,16:30:00,130.0,Retiro Ii,205517,ICONIC,MALE,GRATUITO,2022-06-15 16:24:00,2022-06-15 16:30:00,POINT (-58.3828403 -34.5970497),POINT (-58.37436403 -34.5917376)


In [116]:
# Example 2: Sort DataFrame by date column in descending order
df_plaza_italia.sort_values(by=['datetime_origen','datetime_destino'], ascending = True, inplace = False) 

AttributeError: 'tuple' object has no attribute 'sort_values'

In [101]:
df_plaza_italia[0].sort_values(by=['datetime_origen','datetime_destinio'])

,Id_recorrido,duracion_recorrido,dia_origen_fecha,dia_semana_origen,horario_origen,id_estacion_origen,nombre_estacion_origen,dia_destino_fecha,dia_semana_destino,horario_destino,id_estacion_destino,nombre_estacion_destino,id_usuario,modelo_bicicleta,Género,tipo_viaje,datetime_origen,datetime_destino,lat_lon_origen,lat_lon_destino
1159223,14731488,459,2022-06-14,3,08:50:00,33.0,Facultad De Medicina,2022-06-14,3,08:58:00,38.0,Plaza Libertad,252459,ICONIC,FEMALE,GRATUITO,2022-06-14 08:50:00,2022-06-14 08:58:00,POINT (-58.3989807 -34.5970909),POINT (-58.3828403 -34.5970497)
1159234,14737402,846,2022-06-14,3,17:46:00,38.0,Plaza Libertad,2022-06-14,3,18:00:00,116.0,Hospital Alemán,207946,ICONIC,OTHER,GRATUITO,2022-06-14 17:46:00,2022-06-14 18:00:00,POINT (-58.3828403 -34.5970497),POINT (-58.4025894 -34.5921708)
1159374,14759176,3011,2022-06-16,5,16:17:00,38.0,Plaza Libertad,2022-06-16,5,17:07:00,187.0,Jose Maria Moreno,32558,ICONIC,MALE,GRATUITO,2022-06-16 16:17:00,2022-06-16 17:07:00,POINT (-58.3828403 -34.5970497),POINT (-58.435801 -34.619879)
1159392,14760046,1401,2022-06-16,5,17:13:00,137.0,Azopardo Y Chile,2022-06-16,5,17:37:00,38.0,Plaza Libertad,16035,ICONIC,MALE,GRATUITO,2022-06-16 17:13:00,2022-06-16 17:37:00,POINT (-58.3674923 -34.6155977),POINT (-58.3828403 -34.5970497)
1159687,14726525,751,2022-06-13,2,17:55:00,38.0,Plaza Libertad,2022-06-13,2,18:07:00,116.0,Hospital Alemán,207946,ICONIC,OTHER,GRATUITO,2022-06-13 17:55:00,2022-06-13 18:07:00,POINT (-58.3828403 -34.5970497),POINT (-58.4025894 -34.5921708)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1347347,14726349,793,2022-06-13,2,17:43:00,96.0,Carlos Gardel,2022-06-13,2,17:56:00,38.0,Plaza Libertad,674273,FIT,OTHER,GRATUITO,2022-06-13 17:43:00,2022-06-13 17:56:00,POINT (-58.4116586 -34.6027814),POINT (-58.3828403 -34.5970497)
1347383,14750105,556,2022-06-15,4,18:53:00,30.0,Peña,2022-06-15,4,19:03:00,38.0,Plaza Libertad,674273,FIT,OTHER,GRATUITO,2022-06-15 18:53:00,2022-06-15 19:03:00,POINT (-58.3973698 -34.5908211),POINT (-58.3828403 -34.5970497)
1347863,14755293,550,2022-06-16,5,10:47:00,38.0,Plaza Libertad,2022-06-16,5,10:56:00,33.0,Facultad De Medicina,553506,FIT,OTHER,GRATUITO,2022-06-16 10:47:00,2022-06-16 10:56:00,POINT (-58.3828403 -34.5970497),POINT (-58.3989807 -34.5970909)
1348209,14722656,1456,2022-06-13,2,13:03:00,38.0,Plaza Libertad,2022-06-13,2,13:27:00,9.0,Parque Las Heras,643759,FIT,OTHER,GRATUITO,2022-06-13 13:03:00,2022-06-13 13:27:00,POINT (-58.3828403 -34.5970497),POINT (-58.407741 -34.585443)
